In [1]:
from dotenv import load_dotenv

In [2]:
load_dotenv()

True

In [3]:
from langchain_openai import OpenAI

In [4]:
llm = OpenAI(temperature=0.6)

In [5]:
name = llm.invoke('I want to open a restorant for Indian food. Suggest a fancy name for this.')
print(name)



"Spice Palace: A Taste of India"


In [6]:
from langchain.prompts import PromptTemplate

In [7]:
prompt_template_name = PromptTemplate(
    template='I want to open a restorant for {cuisine} food. Suggest a fancy name for this.', 
    input_variables=['cuisine']
)

prompt_template_name.format(cuisine='Indian') 

'I want to open a restorant for Indian food. Suggest a fancy name for this.'

In [8]:
from langchain.chains import LLMChain

In [9]:
name_chain = LLMChain(llm=llm, prompt=prompt_template_name)

C:\Users\prith\AppData\Local\Temp\ipykernel_12056\3541426536.py:1: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use :meth:`~RunnableSequence, e.g., `prompt | llm`` instead.
  name_chain = LLMChain(llm=llm, prompt=prompt_template_name)


In [10]:
name = name_chain.invoke('Chinees')
print(name)

{'cuisine': 'Chinees', 'text': '\n"Golden Dragon Palace"'}


In [11]:
name_chain = prompt_template_name | llm

In [12]:
name = name_chain.invoke('Chinees')
print(name)



"Imperial Dragon Palace"


In [13]:
prompt_template_name = PromptTemplate(
    template='I want to open a restorant for {cuisine} food. Suggest a fancy name for this.', 
    input_variables=['cuisine']
)

prompt_template_items = PromptTemplate(
    template='Suggest some menu items for {restaurant_name}. Return it as a comma separated list.',
    input_variables=['restaurant_name']
)

In [14]:
chain = prompt_template_name | prompt_template_items | llm

In [15]:
print(chain.invoke("Indian"))



1. Tandoori Palace 
2. Spice Kingdom 
3. Curry Mansion 
4. Masala Mansion 
5. Naan Nook 
6. Tandoori Terrace 
7. Saffron Palace 
8. Chai Chalet 
9. Biryani Bistro 
10. Samosa Spot


In [16]:
from dotenv import load_dotenv
load_dotenv()

from langchain_openai import OpenAI
from langchain.prompts import PromptTemplate
from langchain_core.runnables import RunnableMap, RunnableLambda

llm = OpenAI(temperature=0.6)

# Prompt to generate restaurant name
prompt_template_name = PromptTemplate(
    template='I want to open a restaurant for {cuisine} food. Suggest a fancy name for this.',
    input_variables=['cuisine']
)

# Prompt to generate menu items
prompt_template_items = PromptTemplate(
    template='Suggest some menu items for {restaurant_name}. Return it as a comma separated list.',
    input_variables=['restaurant_name']
)

# Chain 1: cuisine -> restaurant_name
name_chain = RunnableMap({
    "restaurant_name": prompt_template_name | llm
})

# Custom chain to keep restaurant_name and generate menu_items
def generate_menu_with_name(inputs):
    restaurant_name = inputs["restaurant_name"]
    menu_items = (prompt_template_items | llm).invoke({"restaurant_name": restaurant_name})
    return {
        "restaurant_name": restaurant_name,
        "menu_items": menu_items
    }

# Full chain: cuisine -> restaurant_name -> menu_items
full_chain = name_chain | RunnableLambda(generate_menu_with_name)

# Invoke chain
result = full_chain.invoke({"cuisine": "Mexican"})
print("Restaurant Name:", result["restaurant_name"])
print("Menu Items:", result["menu_items"])

Restaurant Name: 

"El Sabor de México" (The Flavor of Mexico) 
Menu Items: 

1. Tacos al Pastor
2. Enchiladas Verdes
3. Carne Asada
4. Chiles Rellenos
5. Pozole
6. Fajitas
7. Carnitas
8. Quesadillas
9. Tamales
10. Sopes
11. Mole Poblano
12. Chimichangas
13. Tostadas
14. Burritos
15. Flautas
16. Ceviche
17. Chilaquiles
18. Tres Leches Cake
19. Horchata
20. Margaritas


In [17]:
from dotenv import load_dotenv
load_dotenv()

True

In [18]:
from langchain_openai import OpenAI
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain, SequentialChain

In [19]:
llm = OpenAI(temperature=0.6)

In [20]:
prompt_template_name = PromptTemplate(
    template='I want to open a restaurant for {cuisine} food. Suggest a fancy name for this.',
    input_variables=['cuisine']
)

name_chain = LLMChain(llm=llm, prompt=prompt_template_name, output_key='restaurant_name')

In [21]:
prompt_template_items = PromptTemplate(
    template='Suggest some menu items for {restaurant_name}. Return it as a comma separated list.',
    input_variables=['restaurant_name']
)

menu_chain = LLMChain(llm=llm, prompt=prompt_template_items, output_key='menu_items')

In [23]:
chain = SequentialChain(
    chains = [name_chain, menu_chain], 
    input_variables=['cuisine'], 
    output_variables=['restaurant_name', 'menu_items']
)

result = chain.invoke({"cuisine": "Mexican"})
print(result)
print("Restaurant Name:", result["restaurant_name"])
print("Menu Items:", result["menu_items"])

{'cuisine': 'Mexican', 'restaurant_name': '\n"El Jardín de Sabores" (The Garden of Flavors)', 'menu_items': '\n\n1. Ensalada de Jardín (Garden Salad)\n2. Fajitas de Pollo con Verduras (Chicken Fajitas with Vegetables)\n3. Filete de Pescado al Limón (Lemon Fish Fillet)\n4. Tacos de Carne Asada (Grilled Steak Tacos)\n5. Arroz con Camarones (Shrimp and Rice)\n6. Quesadillas de Hongos (Mushroom Quesadillas)\n7. Sopa de Tortilla (Tortilla Soup)\n8. Enchiladas de Queso (Cheese Enchiladas)\n9. Tamales de Pollo (Chicken Tamales)\n10. Burrito de Vegetales (Vegetable Burrito)\n11. Flautas de Res (Beef Flautas)\n12. Tostadas de Ceviche (Ceviche Tostadas)\n13. Chiles Rellenos (Stuffed Peppers)\n14. Guacamole y Chips (Guacamole and Chips)\n15. Tres Leches Cake (Three Milk Cake)\n16. Churros con Chocolate (Churros with Chocolate Sauce)'}
Restaurant Name: 
"El Jardín de Sabores" (The Garden of Flavors)
Menu Items: 

1. Ensalada de Jardín (Garden Salad)
2. Fajitas de Pollo con Verduras (Chicken Fajita

In [32]:
from dotenv import load_dotenv
load_dotenv()

from langchain_openai import OpenAI
from langchain.prompts import PromptTemplate
from langchain_core.runnables import RunnableMap, RunnableLambda

# 1) Initialize your LLM
llm = OpenAI(temperature=0.6)

# 2) Build a map that, given `cuisine`, returns `restaurant_name`
prompt_name = PromptTemplate(
    template="I want to open a restaurant for {cuisine} food. Suggest a fancy name for this.",
    input_variables=["cuisine"]
)
name_chain = RunnableMap({"restaurant_name": prompt_name | llm})

# 3) Build a map that, given `restaurant_name`, returns `menu_items`
prompt_menu = PromptTemplate(
    template="Suggest some menu items for {restaurant_name}. Return it as a comma separated list.",
    input_variables=["restaurant_name"]
)
menu_chain = RunnableMap({"menu_items": prompt_menu | llm})

# 4) Wrap both steps in a single RunnableLambda that merges their outputs
def two_step(inputs: dict) -> dict:
    # run the first chain
    name_out = name_chain.invoke(inputs)
    # run the second chain using the first's result
    menu_out = menu_chain.invoke(name_out)
    # merge both dictionaries into one
    return {**name_out, **menu_out}

full_chain = RunnableLambda(two_step)

# 5) Invoke with your initial input
result = full_chain.invoke({"cuisine": "Mexican"})

# 6) Print both pieces cleanly
print("Restaurant Name:", result["restaurant_name"])
print("Menu Items:      ", result["menu_items"])


Restaurant Name: 

"El Jardín de Sabores" (The Garden of Flavors)
Menu Items:       

1. Ensalada de Quinoa (Quinoa Salad)
2. Tacos de Pescado (Fish Tacos)
3. Pollo al Horno con Hierbas (Herb Roasted Chicken)
4. Arroz con Frijoles (Rice and Beans)
5. Sopa de Tortilla (Tortilla Soup)
6. Enchiladas Verdes (Green Enchiladas)
7. Chiles Rellenos (Stuffed Peppers)
8. Carne Asada (Grilled Steak)
9. Camarones al Ajillo (Garlic Shrimp)
10. Fajitas de Vegetales (Vegetable Fajitas)
11. Tamales de Pollo (Chicken Tamales)
12. Flan de Caramelo (Caramel Flan)
13. Horchata (Rice Milk Drink)
14. Margaritas (Traditional Mexican Cocktail)
15. Tres Leches Cake (Milk Soaked Cake)


In [33]:
from langchain_core.runnables import RunnableMap, RunnableLambda
from langchain.prompts import PromptTemplate
from langchain_openai import OpenAI
from dotenv import load_dotenv

load_dotenv()
llm = OpenAI(temperature=0.6)

# Chain A: cooks up a restaurant name
name_chain = RunnableMap({
    "restaurant_name": PromptTemplate(
        template="Suggest a fancy name for a {cuisine} restaurant.",
        input_variables=["cuisine"]
    ) | llm
})

# Chain B: generates menu items
menu_chain = RunnableMap({
    "menu_items": PromptTemplate(
        template="List some signature dishes for '{restaurant_name}'.",
        input_variables=["restaurant_name"]
    ) | llm
})

# Chain C: writes a short tagline
tagline_chain = RunnableMap({
    "tagline": PromptTemplate(
        template="Write a catchy tagline for a restaurant named '{restaurant_name}'.",
        input_variables=["restaurant_name"]
    ) | llm
})

# Wrap all three in one RunnableLambda
def multi_step(inputs: dict) -> dict:
    # 1) name
    out1 = name_chain.invoke(inputs)
    # 2) menu (needs restaurant_name)
    out2 = menu_chain.invoke(out1)
    # 3) tagline (also needs restaurant_name)
    out3 = tagline_chain.invoke(out1)
    # merge all results
    return {**out1, **out2, **out3}

full_chain = RunnableLambda(multi_step)

# Invoke with just the cuisine
result = full_chain.invoke({"cuisine": "Mexican"})
print("Name:   ", result["restaurant_name"])
print("Menu:   ", result["menu_items"])
print("Tagline:", result["tagline"])


Name:    

"La Casa de Sabor" (The House of Flavor)
Menu:    

1. Arroz con pollo (rice with chicken)
2. Ropa vieja (shredded beef stew)
3. Picadillo (ground beef with vegetables)
4. Lechón asado (roast pork)
5. Tostones (fried plantains)
6. Empanadas (stuffed pastries)
7. Paella (Spanish-style rice with seafood and meats)
8. Tamales (corn dough filled with meat and steamed in banana leaves)
9. Mofongo (mashed plantains with garlic and pork cracklings)
10. Churrasco (grilled steak)
11. Camarones al ajillo (garlic shrimp)
12. Pernil (slow-roasted pork shoulder)
13. Flan (caramel custard)
14. Tres leches cake (sponge cake soaked in three types of milk)
15. Moros y cristianos (black beans and rice)
Tagline: 

"Where every bite feels like home."
